# Introduction to Data Science and Systems 2023-2024<small><small>v20232024a</small></small>
## Lab 3: Introduction to Probability
#### - ***you should submit this notebook on Moodle along with one pdf file (see the end of the notebook and Moodle for instructions)***
---
#### University of Glasgow,  JHW (amended by BSJ and NP) 2023

$$
\newcommand{\vec}[1]{ {\bf #1}} 
\newcommand{\real}{\mathbb{R}}
\DeclareMathOperator*{\argmin}{arg\,min}
\newcommand{\expect}[1]{\mathbb{E}[#1]}
$$

## Purpose of this lab

This lab should help you:    
* understand basic probability including probability mass functions and probability density functions
* understand how to manipulate marginal, joint and conditional distributions
* understand Bayes' rule
* understand the multivariate normal distribution
* understand how optimisation can be used to estimate parameters in a simple statistical model using maximum likelihood estimation


## Guide

Lab 3 is structured as follows (with two main task sections):

>- Part 1: Probability with discrete random variables
>- Part 2: Probability with continuous random variables

We recommend you read through the lab *carefully* and work through the tasks.

#### Material and resources 
- It is recommended to keep the lecture notes (from lectures 5 and 6 in particular) open while doing this lab exercise. 
    * ... and you should, of course, be prepared to access some of the recommended material.

#### Marking and Feedback
This assessed lab is marked using three different techniques;

- Autograded with feedback; you'll get immediate feedback.
- Autograded without (immediate) feedback (there will always be a small demo/test so you can be confident that the format of your answer is correct).

*Note*: auto-graded results are always provisional and subject to change in case there are significant issues (this will usually be in favour of the student).

#### Help \& Assistance
- This lab is graded and the lab assistants/lecturer can provide guidance, but we can (and will) not give you the final answer or confirm your result.

#### Plagiarism
- All submissions will be automatically compared against each other to make sure your submission represents an independent piece of work! We have provided a few checks to make sure that is indeed the case.


---

# Before you begin

Please update the tools we use for the automated greading by running the below command (uncomment) and restart your kernel (and then uncomment again) -- or simply perform the installation externally in an Anaconda/Python prompt.

In [ ]:
#!pip install -U --force-reinstall --no-cache https://github.com/johnhw/jhwutils/zipball/master

# the following will allow you to downlad the data files you need (that are otherwise in the zip file)
# you can comment this line after the first run. 
#!pip install wget
#!wget https://github.com/pugeault/IDSS2023-24/raw/main/lab3-files.zip
#!unzip lab4-files.zip

Import the basics...

In [ ]:
# Standard imports
# Make sure you run this cell!
from __future__ import print_function, division
import numpy as np  # NumPy
import scipy.stats 
import os
import sys
import binascii
from unittest.mock import patch
from uuid import getnode as get_mac

from jhwutils.checkarr import array_hash, check_hash, check_scalar, check_string
import jhwutils.image_audio as ia
import jhwutils.tick as tick

###
tick.reset_marks()

# special hash funciton
def case_crc(s, verbose=True):
    h_crc = binascii.crc32(bytes(s.lower(), 'ascii'))
    if verbose:
        print(h_crc)
    return h_crc

# this command generaties a unique key for your system/computer/account
uuid_simple = (("%s") % get_mac())
uuid_str = ("%s\n%s\n%s\n%s\n%s\n") % (os.path,sys.path,sys.version,sys.version_info,get_mac())
uuid_system = case_crc(uuid_str,verbose=False) 

# Set up Matplotlib
import matplotlib as mpl   
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('figure', figsize=(8.0, 4.0), dpi=140)
np.random.seed(2023)


In [ ]:
# Hidden cell for utils needed when grading (you can/should not edit this)

---

**Mini-task**: provide your personal details in two variables:

* `student_id` : a string containing your student id (e.g. "1234567x"), must be 8 chars long.
* `student_typewritten_signature`: a string with your name (e.g. "Adam Smith") which serves as a declaration that this is your own work (read the declaration of originality when you submit on Moodle).

In [ ]:
student_id = "" # your 8 char student id
student_typewritten_signature = "" # your full name, avoid spceical chars if possible

# YOUR CODE HERE
raise NotImplementedError()

---

## Part 1. Hunt the submarine - gridworld version

The USS Scorpion has been lost at sea. Your job is to model where it might be probabilistically. 

<img src="imgs/scorpion.jpg" width="50%">

In part 1 of this lab, we will assume that our world is divided into a 2D grid of squares, each of which might contain an errant submarine.

<img src="imgs/sea_grid.png" width="50%">

### Part 1 Random variables, outcomes and events

We assume that the submarine's location is given by a random variable $X$ whose sample space is the space of 2D grid points $\vec{x}=[x,y]$ where $x$ and $y$ are integers in the range 0-15 (inclusive). (Be careful: the `x,y` order is the opposite of the `row, column` format used in images).

You are given a probability mass function as a 16x16 matrix `submarine_pmf` giving the probability of each grid square, i.e $f_X(\vec{x}) = P(X=\vec{x})$. 


In [ ]:
# import submarine data and functions
from submarine import submarine_pmf, show_pmf

show_pmf(submarine_pmf) # show the PMF on top of a map

### Outcomes and events
Compute the the following values, all of which are scalars. The notation $[x_1, y_1] \rightarrow [x_2, y_2]$ indicates a box that should **include** $x_1,y_1,x_2,y_2$ (be careful!).

* `in_6_2` probability submarine is in square $[6,2]$
* `out_5_5` probability submarine is *not* in square $[5,5]$
* `west_4` submarine is in square $[x\geq 4, y]$
* `y_div_3` submarine has a y coordinate exactly divisible by 3
* `below_above` submarine is in square $[x,y<4] or [x,y\geq 10]$
* `box_1` submarine is in the box $[3, 4] \rightarrow [5, 6]$ (inclusive!)
* `box_2` submarine is in the box $[0, 0] \rightarrow [15, 15]$ (inclusive!)
* `evens` submarine has $x\geq 4$ and an *even* y coordinate
* `odds_not_box` the *odds* that the submarine is *not* in the box $[2, 4] \rightarrow [6,10]$
* `odds_even` the *odds* that the submarine has an x coordinate which is *even*
* `logit_4_6` the *log-odds (logits)* that the submarine is in square $[4, 6]$
* `dlogit_box` the *change* in log-odds from the hypothesis that the submarine is in the box $[1,1] \rightarrow [5, 5]$ to being in the box $[10,10] \rightarrow [15,15]$



In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with tick.marks(1):
    assert check_scalar(in_6_2, '0x98d4201e')


In [ ]:
with tick.marks(1):
    assert check_scalar(out_5_5, '0xddb3de98')

In [ ]:
with tick.marks(1):
    assert check_scalar(west_4, '0x1d40485b')

In [ ]:
with tick.marks(1):
    assert check_scalar(y_div_3, '0xdaa43a7a')

In [ ]:
with tick.marks(1):
    assert check_scalar(below_above, '0xdbedd8bb')

In [ ]:
with tick.marks(1):
    assert check_scalar(box_1, '0x64edb4c9')

In [ ]:
with tick.marks(1):
    assert check_scalar(box_2, '0xb44c37ea')

In [ ]:
with tick.marks(1):
    assert check_scalar(evens, '0xe040e314')

In [ ]:
with tick.marks(1):
    assert (check_scalar(odds_not_box, '0xccfc0501'))
    

In [ ]:
## Hidden test checking odds_even [1 marks]

In [ ]:
with tick.marks(1):
    assert (check_scalar(logit_4_6, '0x2e63d2e9'))


In [ ]:
# Hidden test checking dlogit_box [2 marks]

---

### Expected value
You need to plan for the search operation. There is a fixed search station at square $x=3, y=5$. Assume the Euclidean ($L_2$) norm in measuring distances. The time $t$ in hours to search a grid square from the station is given by $t = d^2 + 4d + 5$. Compute:

* `expected_location` Expected value of the submarine location.
* `expected_distance` expected value of the distance of the submarine to this fixed station 
* `total_search_time` the **total** search time required to search the every square on the entire grid. Assume each search of a grid square has to restart from the station.
* `expected_search_time` expected search time of a random search starting at the station.

**REMEMBER**, in general $$E[f(X)] \neq f(E[X]).$$

In [ ]:
show_pmf(submarine_pmf)
plt.plot(3,5, 'C2o', label='Search station', markersize=10)
plt.legend(loc='lower left');

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with tick.marks(2):
    assert check_scalar(expected_location[0], '0x4f9f5e7c')
    assert check_scalar(expected_location[1], '0x8805a495')


In [ ]:
with tick.marks(2):
    assert check_scalar(expected_distance, '0x8b8d5382')

In [ ]:
with tick.marks(2):
    assert check_scalar(total_search_time, '0x74a48a1')

In [ ]:
# Hidden test checking expected_search_time [3 marks]

---

### Part 1.3 Conditional probability
The search strategy could be improved if one of the $x$ or $y$ coordinates were known for sure (e.g. from another source, like satellite imaging). To identify how *much* this would help, we can use conditional probability distributions. Compute the following:


* `p_x_y` the *conditional* PMF of an finding the submarine on an x coordinate given a y coordinate $P(X_x=x|X_y=y)$ as 16x16 matrix
* `p_y_x` the *conditional* PMF of finding the submarine on a y coordinate given an x coordinate $P(X_y=y|X_x=x)$ as 16x16 matrix
* `p_y_7` the conditional PMF of the submarine being in squares with y=7, as a single 16 element vector.
* `p_even_x` the PMF of the submarine being on a given y coordinate if we know the submarine is in a grid square with even x coordinate, as a single 16 element vector. $P(X_y=y|X_x = x, x\ \text{even})$
* `p_even_y_odd_x` the PMF of the submarine being on an even y coordinate if we know the submarine is in a grid square with odd x coordinate, as a single **8 element** vector. $P(X_y=y|X_x = x, x\ \text{odd}, y\ \text{even})$





In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with tick.marks(2):    
    show_pmf(p_x_y, "P(x|y)")
    assert check_hash(p_x_y, ((16, 16), 2103.534977619737))    

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with tick.marks(2):
    show_pmf(p_y_x, "P(y|x)")
    assert check_hash(p_y_x, ((16, 16), 2031.7414411989753))    

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with tick.marks(2):
    show_pmf(p_y_7[:,None], 'p(x|y=7)')
    assert check_hash(p_y_7, ((16,), 8.99944095882301 ))    

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with tick.marks(2):
    show_pmf(p_even_x[None,:], 'p(y|x even)')
    assert check_hash(p_even_x, ((16,), 7.1624313253524745))

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
#Sanity check ; just to check size
with tick.marks(0):
    assert check_hash(0.0*p_even_y_odd_x, ((8,), 0.0 ))

tmp_pmf = np.stack([p_even_y_odd_x, 0*p_even_y_odd_x]).T.reshape(16,)    
show_pmf(tmp_pmf[None,:], 'p(y even|x odd)')

In [ ]:
# Hidden test checking p_even_y_odd_x [3 marks]

---

### Entropy
The search could be guided more intelligently if we could work out how informative getting one of the coordinates could be. For example, knowing which `y` coordinate would give you *most* information about the `x` coordinate?  Compute the entropy of each conditional distribution $H(Y=n|X)$ and store it in `entropy_y`. Use base 2 for the entropy (i.e. bits).
Store the most useful `y` coordinate to know in `most_informative_y`.


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with tick.marks(5):
    show_pmf(entropy_y[:, None])
    assert check_hash(entropy_y, ((16,), 541.0644437253504))

In [ ]:
#Sanity check; just to check size
with tick.marks(0):
    assert check_hash(0.0*most_informative_y, ((), 0.0 ))

In [ ]:
# Hidden test checking most_informative_x [5 marks]

---

### Part 1.4 Validating pmfs

`submarine_pmf` is one possible model of the submarine location. A number of others have been proposed for this 16x16 gridworld model of searching by the scientific reserach team; but some of them are implemented incorrectly and are not valid PMFs. These probability mass functions are stored in a list of matrices 16x16 `proposed_pmfs`. Identify which of these PMFs are *valid* PMFs and store the valid ones in a list `valid_pmfs`.


In [ ]:
from submarine import proposed_pmfs

# show each of the PMFs
for i,pmf in enumerate(proposed_pmfs):
    show_pmf(pmf, title=f"PMF {i}")


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with tick.marks(3):
    check_hash(valid_pmfs, ((2, 16, 16), 511.54549199055594 ))

---

## Sampling
We can use sampling to simulate where the submarine might lie on the sea floor. Write a function `sample_submarine(n)` which will randomly sample submarine locations according to `submarine_pmf`, returning a `n x  2` array of sampled locations. Hint: there are 256 possible grid squares.

Note: your code may pass these tests but be incorrect; this will become apparent when you attempt the "reconstruct the PMF" part below.

In [ ]:
def sample_submarine(n, pmf=submarine_pmf):
# YOUR CODE HERE
raise NotImplementedError()

# example of how to call your sample_submarine function
sample_submarine(100)

In [ ]:
with tick.marks(3):    
    for n in [2, 4, 10, 1000, 5000]:
        samples =  sample_submarine(n)
        assert samples.shape == (n,2)
        assert np.sum(samples - np.floor(samples)) == 0.0
        mean = np.mean(samples, axis=0)
        sem = np.std(samples, axis=0)/np.sqrt(n)    
        assert np.all((mean-sem < [7.93, 5.67]) | ([7.93, 5.67] < mean+sem))

---

### Part 1.6 Reconstruct the PMF

From our samples (more realistically samples would be coming from an other source), we can reconstruct an approximation of the PMF. Write a function `reconstruct_pmf(samples)` that will reconstruct the PMF using the empirical distribution from a set of samples and return the reconstructed PMF as a 16x16 matrix. 


In [ ]:
def reconstruct_pmf(samples):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
def kl(a, b):
    eps = 1e-9 # small constant used to avoid division by zero
    return np.sum(np.where((b<1e-10)|(a<1e-10), 0.0, a * np.log((a+eps)/(b+eps))))

with tick.marks(2):    
    kls = [4.0, 3.5, 2.5, 0.1, 0.02]
    for i,n in enumerate([1, 4, 10, 1000, 5000]):        
        approx_pmf =  reconstruct_pmf(sample_submarine(n))
        assert approx_pmf.shape == submarine_pmf.shape
        assert check_scalar(np.sum(approx_pmf), "0xb44c37ea")     
        assert np.max(approx_pmf)<=1.0
        assert np.min(approx_pmf)>=0.0
        kl_div = abs(kls[i] - kl(approx_pmf, submarine_pmf))/kls[i]        
        assert kl_div < 1.0             
        sample = reconstruct_pmf(sample_submarine(n))
        show_pmf(sample, f"{n} samples")             

---

### Part 1.7 Log likelihood
Various theoretical models of oceanographic features have been used to simulate what happened to the submarine. 

* crater
* zeeman_shift
* oblov_1
* minority
* inviscid

Each of these models produces a collection of simulated submarine locations as a result.

`submarine_samples` provides a dictionary with samples from various named submarine position generating functions. Compute the *log*-likelihood of each sequence $\log \mathcal{L}(x_1, x_2,\dots)$ under the model defined by the PMF `submarine_pmf`. 
* `sequence_lliks`  a dictionary that maps the names to the log-likelihood.
* `most_likely_sequence` the name of the sequence is most likely to be compatible with this model.


In [ ]:
from submarine import submarine_samples

print(submarine_samples.keys())

In [ ]:
def llik(seq):
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:


hashes = {
    "crater": "0x7315af59",
    "zeeman_shift": "0x2c18773a",
    "oblov_1": "0x11b0e54d",
    "minority": "0x37d528a8",
    "inviscid": "0x6bb2ae44",
}

with tick.marks(3):
    for key in submarine_samples.keys():
        print(key, sequence_lliks[key])
        assert check_scalar(sequence_lliks[key], hashes[key])

In [ ]:
# Hidden test checking the string most_likely_sequence [1 mark]

---

### Part 1.8 Bayes' rule

The function `search_submarine(x,y)` returns the **likelihood** for the submarine's location, given an `x` and `y` coordinate to search at based on sonar returns from a search ship. It returns this as a 16x16 matrix.

Using `submarine_pmf` as a prior, compute the posterior PMF using the result of `search_submarine(10, 7)` as likelihood. Store the result in `submarine_posterior`.

In [ ]:
from submarine import search_submarine

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
show_pmf(submarine_pmf, 'Prior')
show_pmf(search_submarine(10,7), 'Likelihood' )
show_pmf(submarine_posterior, 'Posterior')
with tick.marks(5):
    assert check_hash(submarine_posterior, ((16, 16), 129.2249265521699))

---

### Part 1.9 Sequential searching 
Compute the posterior distribution after searching the each of squares y=5, x=0..15 (i.e. testing *each* of these 16 squares in turn, and combining *all* of the evidence into a posterior), storing the posteriors as a list, *starting with the prior* and ending with the posterior after observing y=5, x=15. You should have a 17 element list of 16x16 matrices. Store this as `search_strip_posterior`


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Sanity check that the size is ok [0 marks]
with tick.marks(0):
    assert(len(search_strip_posterior)==17)
    assert(search_strip_posterior[0].shape==(16, 16)) # only check one of the 17 matricies

# Show the 17 individual posteriors
for i, p in enumerate(search_strip_posterior):
    show_pmf(p, f'Posterior after searching y=5, x={i}')


In [ ]:
# Hidden test checking search_strip_posterior [6 marks ]

---

---


---

---

## Part 2. Hunt the submarine: continuous version
The simple discrete grid model is easy to do computations with but not a very realistic model. A better world model is that locations are *continuous* in nature; for example points specified as longitude and latitude. To work with this model probabilistically, we need to use (multivariate, i.e. vector) **probability density functions** which map coordinates to *density*, not probability.



### Multivariate normal distribution
The PDF for the multivariate Normal distribution is defined as follows (for column vectors):

$$f_{{X}}(\vec{x})=\frac{1}{\sqrt{(2\pi)^d|\boldsymbol\Sigma|}}\operatorname{exp}\left( -\frac{1}{2} (\vec{x}-\boldsymbol\mu)^\top \boldsymbol\Sigma^{-1} (\vec{x}-\boldsymbol\mu)   \right)$$
where $\boldsymbol\mu$ is the mean (vector), $d$ is the dimension of $\vec{x}$ and $\boldsymbol\Sigma$ is the $d \times d$ covariance matrix.

### Continuous distributions
A continuous random variable can be represented as a class that has parameters and that provides two functions:

* `llik` the log-likelihood of an observation
* `sample` that returns a sample from a distribution

The multivariate normal PDF is implemented below using standard `scipy` implementations along with useful methods for drawing the density for debugging.

In [ ]:
import scipy.stats
from jhwutils.ellipse import _cov_ellipse

class MultivariateNormal:
    
    def __init__(self, mu, sigma):
        # use the existing implementation in scipy.stats
        self.mu = mu
        self.sigma = sigma
        self.distribution = scipy.stats.multivariate_normal(self.mu, self.sigma)
    
    # log-likelihood of one observation
    def llik(self, x):
        return self.distribution.logpdf(x)
    
    # n random samples
    def sample(self, n):
        return self.distribution.rvs(n)        
        
    # render the distribution pdf as if it were a PMF
    def draw(self):            
        span = np.linspace(0,16,160)
        grid = np.array(np.meshgrid(span, span)).reshape(2, -1).T        
        pdf = self.distribution.pdf(grid)        
        show_pmf(pdf.reshape(160,160).T)                        

---

### Part 2.1
Instantiate a multivariate Normal distribution using the `MultivariateNormal` class defined above. Store it in `x`. E.g. `x = MultivariateNormal(mu=m, sigma=S)`

The PDF should have mean with coordinates (6,6), variance of one along both the latitude and longitude direction, and a covariance of 0.6.


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Autograded [2 marks]

with tick.marks(2):
    assert check_hash(x.mu, ((2,), 42.0))
    assert check_hash(x.sigma, ((2,2), 11.4))

**Try varying these parameters (in a new unassessed cell) and viewing the resulting distributions to understand what effect they have:**

---

### Part 2.2 Fitting with direct computation

There are a collection of *observed* potential locations for the submarine from a search ship in `submarine_observations`. The problem is to fit the parameters of a PDF given these observations.

Recall that the **arithmetic mean** is sum of sample values $\vec{x}_1, \vec{x}_2, \dots, \vec{x}_n$ divided by the number of values:
$$\hat{\boldsymbol\mu} = \frac{1}{n} \sum_{i=1}^n \vec{x}_i$$

The empirical covariance is computed by:

$${\boldsymbol{\hat \Sigma }} = \frac{1}{{n - 1}}\sum\limits_{i = 1}^n {{{\left( {{{\mathbf{x}}_i} - {\boldsymbol{\hat \mu }}} \right)}^T}\left( {{{\mathbf{x}}_i} - {\boldsymbol{\hat \mu }}} \right)} $$

Under the assumption that a reasonable model for the submarine location is a multivariate normal:

* directly estimate the parameters for a `MultivariateNormal` using standard numpy functions (which we have used in previous labs and several lectures) and store the result in `normal_mu_hat` and `normal_sigma_hat`.
* `llik_normal` the log-likelihood of the *entire* set of observations under this model (i.e. with these parameters)
* `samples_normal` 250 samples drawn from the multivariate normal model with these parameters
* `llik_misfit` how much smaller the log-likelihood of observations is compared to the samples *generated under the model* (i.e. `samples_normal`). This should be a negative number.

In [ ]:
submarine_observations = np.loadtxt('data/submarine2023.txt')

In [ ]:
# Set a seed - DO NOT MODIFY
np.random.seed(2023)
############################

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
x = MultivariateNormal(normal_mu_hat, normal_sigma_hat)

x.draw()
ax = plt.gca()
ax.scatter(samples_normal[:,0], samples_normal[:,1], c='r', marker='x', s=1, label="Samples")
ax.scatter(submarine_observations[:,0], submarine_observations[:,1], marker='o', color='w', s=2, label="Observations")
ax.set_xlim(0,16)
ax.set_ylim(0,16)
ax.legend()
ax.set_title(f"Log-likelihood = {llik_normal:.2f}")

In [ ]:
with tick.marks(2):
    assert check_hash(normal_mu_hat, ((2,), 48.304595497516765))
    assert check_hash(normal_sigma_hat, ((2,2), 16.421940503564702))    

In [ ]:
with tick.marks(2):
    assert samples_normal.shape == (250, 2)
    assert np.all(np.std(samples_normal, axis=0)>0.5)
    assert np.all(np.mean(samples_normal, axis=0) - np.array([8, 6]) < 4.0 * (np.std(samples_normal, axis=0)/np.sqrt(250)))  

In [ ]:
# Hidden test checking the scalar llik_normal [3 marks]

In [ ]:
# Hidden test checking the scalar llik_misfit [4 marks]

---

### Part 2.3: A cross search pattern

Scientists argue that a better model for the location of the submarine is a cross-shaped density that reflects two possible ocean currents in the Atlantic and is also easier to search by ship. The scientists have written a probabilistic model which gives the density of a specialised "cross distribution". This is a non-standard distribution that has four parameters: `ctr`, `span`, `leg` and `angle`.

<img src="imgs/ship_cross.png" width="50%">


In [ ]:
class CrossDistribution:
    def __init__(self, ctr, span, leg, angle):
        self.ctr = ctr
        leg = 1.0 + np.log(leg + 1.0)
        s, c = np.sin(np.radians(angle)), np.cos(np.radians(angle))
        r1 = np.array([[s / leg, c * leg], [c / leg, -s * leg]])
        r2 = np.array([[-c / leg, s * leg], [s / leg, c * leg]])
        scale = np.eye(2) * np.sqrt(span)
        self.arm1 = scipy.stats.multivariate_normal(self.ctr, r1 @ scale @ scale @ r1.T)
        self.arm2 = scipy.stats.multivariate_normal(self.ctr, r2 @ scale @ scale @ r2.T)

    # log-likelihood of one observation
    def llik(self, x):
        return np.log((self.arm1.pdf(x) + self.arm2.pdf(x)) / 2.0)

    # n random samples
    def sample(self, n):
        a1_samples = self.arm1.rvs(n)
        a2_samples = self.arm2.rvs(n)
        return np.where(np.random.uniform(0, 1, n) < 0.5, a1_samples.T, a2_samples.T).T

    # render the distribution pdf as if it were a PMF
    def draw(self):
        span = np.linspace(0, 16, 160)
        grid = np.array(np.meshgrid(span, span)).reshape(2, -1).T

        pdf = 0.5 * self.arm1.pdf(grid) + 0.5 * self.arm2.pdf(grid)
        show_pmf(pdf.reshape(160, 160).T)

**Try varying these parameters and viewing the resulting distributions to understand what effect they have:**

In [ ]:
cf = CrossDistribution(ctr=(7,5), span=5.1, leg=3.0, angle=-30.)
cf.draw()

---

### Part 2.4 Expectation

We would like to be able to compute the expected location (and search time/distance) for this model. But we can't do a simple operation with a PDF like we could with the PMF model. Instead, we can use sampling methods to approximate the expectation $\hat{\mathbb{E}}[f(X)]$.

Using the same formula $t = d^2 + 4d + 5$ as in Part 1, use the Monte Carlo approximation to the expectation, with 1000 samples, to compute:

* `cross_expected_location` the expected location  (a 2D vector)
* `cross_expected_distance` expected L2 distance from the fixed station at (14,2) 
* `cross_expected_time`  expected search time in hours from this station.

with a cross distribution having the parameters:
    
        ctr = (7,10)
        span = 0.5
        leg = 7.0
        angle = 45.0


In [ ]:
# Seed
np.random.seed(2023) # DO NOT CHANGE
##########################

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
with tick.marks(1):
    assert check_scalar(np.sum(cross_expected_location), '0xba7e9b1', tol=1)


In [ ]:
with tick.marks(2):
    assert check_scalar(cross_expected_distance, '0x84e71c11', tol=1)

    

In [ ]:
# Hidden test cross_expected_time [5]

---

## Part 2.5 Estimating parameters by optimising the likelihood 
*Note* this is the difficult and challenging part of this lab.

There is no direct form to compute the parameters  `ctr`, `span`, `leg` and `angle` from `submarine_observations` in the way we could for the simple normal distribution. 

We can instead exploit the idea of maximum likelihood which aims to make inferences abou tthe parameters that where most likely to generate the observations. Recall that we have the likelihood of a single observation under the model

$$\mathcal L(\theta;\vec{x}) = f_X(\vec{x};\theta)$$

where $\theta$ are the parameters `ctr`, `span`, `leg` and `angle` and $\vec{x}$ is one of the 2d observations in `submarine_observations`.

If we assume our observations are independent and identically distributed, we can define an **objective function** for all observations, namely the product of the individual likelihood terms

$$\mathcal{L}(\theta ;{\{ {x_i}\} ^{i = 1:n}}) = -\prod\limits_{i = 1}^n {\mathcal{L}(\theta ;{x_i})} $$

If we take the log (perhaps to achieve greater numerical stability) and futher more take the negative value (as maximiseing the likelihood, is equivalent to minimiseing the negative likelihood), we get the following optimisation problem:

$$\theta^* = \argmin_\theta L(\theta)$$
$$L(\theta) = -\log \mathcal{L}(\theta | x_1, \dots, x_n) = -\sum_i \log f_X(x_i;\theta) , $$

This is *very* similar to the approximation objective function we saw in the optimisation part of this course $\|f(\vec{x};\theta)-y\|$, but we have $y=0$ and we only have a scalar output from $f$ so the norm is unnecessary. We already know how to solve this kind of problem; just optimise. This is called **maximum likelihood estimation** and is a general technique for determining parameters of a distribution which we don't know given some observations. It will find the **best** setting of parameters that would explain how the observations came to be.


Using your own custom  **stochastic hill-climbing** (you can find code in the optimisation lab) and the idea of **maximum likelihood**, create a function `optimise_llik` to find the best parameter settings `theta= [ctr_x, ctr_y, span, leg, angle]` given the observations in `submarine_observations`. The signature of the function is given in the cell below.

In the specific setting we also know that; 

* `ctr` must be a vector in [0,0] to [16,16], 
* `span` must be a scalar in range 0.0 to 20.0
* `leg` must be a scalar from 0.0 to 10.0
* `angle` must be a scalar from 0.0 to 45.0 

which must be accounted for in your optimisation routine to find a best-fit estimate for `theta`. **Solutions not obeying the constraints will result in zero marks.**

You should be able to find a good fit within 10,000 iterations (max 15000). You will need to modify the stochastic hill climbing code to enforce the above constraints and ensure that you algorithm converges to a sensible solution by inspecting the plots and parameters. You may need to change the random seed and various parameters to arrive at a sensible solution.

**Note: We will test your solution on multiple instances so you cannot hardcode the solution. We will NOT accept solutions using any in-built optimization functions.**


In [ ]:
# Use this cell for implementing all your helper functions 

# YOUR CODE HERE
raise NotImplementedError()


In [ ]:
def optimise_llik(observations,pdf,iters):
    """
        observations: a numpy array with the observations (e.g submarine_observations)
        pdf: the PDF (e.g. CrossDistribution)
        iters: number of iterations   
        
        Note: you would likely need to implement a number of helper functions in the cell above.
    """
    # YOUR CODE HERE
    raise NotImplementedError()
    return theta # return the found parameters


Run the implemented optimisation function on the submarine_observations using the CrossDistribution. You can change the random seed and number of iterations (**max 15000 iterations**!).

In [ ]:
# Note these values will be used in the test so make sure 
# they reflect you ideal configuration upon submission
my_random_seed = 2019
iters = 15000
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
np.random.seed(my_random_seed) # you are free to change this if helpful in the previous cell
theta = optimise_llik(observations=submarine_observations, pdf=CrossDistribution, iters=iters)
print("Estimated parameters:", theta)


Let us plot the solution. **Hint**: While you don't know the true parameters you can do various inspections/tests to ensure that your solution is sensible before submitting.  

In [ ]:
cf = CrossDistribution(ctr=(theta[0],theta[1]), span=theta[2], leg=theta[3], angle=theta[4])
cf.draw()
ax = plt.gca()
ax.scatter(submarine_observations[:,0], submarine_observations[:,1], marker='o', color='w', s=2, label="Observations")
ax.set_xlim(0,16)
ax.set_ylim(0,16)
fig_2_5 = plt.gcf() # do not modify


In [ ]:
# Hidden auto graded [5 marks]... if your optimiser gets you in the vicinity of the true parameters 

In [ ]:
# Hidden auto graded [8 marks]... if your optimiser gets you very close to the true parameters

In [ ]:
# A hidden cell for use by markers


---


# Submission on Moodle


We will generate the **one** pdf file you'll need to submit along with the notebook:

*Note*: you do not need to worry about the formatting etc (that's predetermined)


In [ ]:
## Report generation - YOU MUST YOU RUN THIS CELL !
#
# Ignore warnings regarding fonts
#
from matplotlib.backends.backend_pdf import PdfPages

# Declaration of originality with system info
try:
    f = open('uofg_declaration_of_originality.txt','r')
    uofg_declaration_of_originality = f.read()
except: 
    uofg_declaration_of_originality = "uofg_declaration_of_originality not present in cwd"

try:
    student_id.lower()
except: 
    student_id="NORESPONSE"
try:
    student_typewritten_signature.lower()
except: 
    student_typewritten_signature="NORESPONSE"

fn = ("idss_lab3_probability_%s_declaration.pdf" % (student_id.lower()))
fig_dec = plt.figure(figsize=(10, 12)) 
fig_dec.text(0.1,0.1,("%s\n\n Student Id %s\n\n Typewritten signature: %s\n\n UUID System: %s" % (uofg_declaration_of_originality,student_id, student_typewritten_signature, uuid_system)))
    
# Combined: 
fn = ("idss_lab3_probability_%s_combined_v20232024a.pdf" % (student_id))
pp = PdfPages(fn)
pp.savefig(fig_2_5)
pp.savefig(fig_dec)
pp.close()

with tick.marks(0):  # have you generated the combined file...? you don't actually get any credit for this; just confirmation that the file has been generated
    assert(os.path.exists(fn))

**You must (for full or partial marks) submit via Moodle:**

- this notebook (completed) after "Restart and rerun all":
    - `idss_lab3_probability_v20232024a.ipynb`
    
- the combined pdf (autogenerated) containing the relevant figures and answers for the manual marking:
     - `idss_lab3_probability_[YOUR STUDENT ID]_combined_v20232024a.pdf`)     


---

# Appendix: Marking Summary (and other metadata)
#### - make sure the notebook runs without errors (remove/resolve the `raise NotImplementedError()`) and "Restart and Rerun All" cells to get a correct indication of your marks.

In [ ]:
print("Marks total : ","100")
print("Marks visible (with immediate feedback): ","54")
print("Marks hidden (without immediate feedback): ","46")
print("\nThe fraction below displays your performance on the autograded part of the lab that's visible with feedback (only valid after `Restart and Run all`:")
tick.summarise_marks() # 
print("- the autograded (and visible) marks account for at least 50% of the total lab assesment.")

---